<a href="https://colab.research.google.com/github/LiviaBBLeap/motionblur/blob/main/wavelet_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Per decides whether the image is blured or not based on the lack of a type of edges (they dissapear w haar wavelet transformation), blurred extent give the sharpness of the edges left(other type)

In [ ]:
"""
@author: pedrofRodenas
"""

import pywt
import cv2
import numpy as np
import os
import argparse
import json

def blur_detect(img, threshold):
    
    # Convert image to grayscale
    Y = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
    M, N = Y.shape
    
    # Crop input image to be 3 divisible by 2
    Y = Y[0:int(M/16)*16, 0:int(N/16)*16]
    
    # Step 1, compute Haar wavelet of input image
    LL1,(LH1,HL1,HH1)= pywt.dwt2(Y, 'haar')
    # Another application of 2D haar to LL1
    LL2,(LH2,HL2,HH2)= pywt.dwt2(LL1, 'haar') 
    # Another application of 2D haar to LL2
    LL3,(LH3,HL3,HH3)= pywt.dwt2(LL2, 'haar')
    
    # Construct the edge map in each scale Step 2
    E1 = np.sqrt(np.power(LH1, 2)+np.power(HL1, 2)+np.power(HH1, 2))
    E2 = np.sqrt(np.power(LH2, 2)+np.power(HL2, 2)+np.power(HH2, 2))
    E3 = np.sqrt(np.power(LH3, 2)+np.power(HL3, 2)+np.power(HH3, 2))
    
    M1, N1 = E1.shape

    # Sliding window size level 1
    sizeM1 = 8
    sizeN1 = 8
    
    # Sliding windows size level 2
    sizeM2 = int(sizeM1/2)
    sizeN2 = int(sizeN1/2)
    
    # Sliding windows size level 3
    sizeM3 = int(sizeM2/2)
    sizeN3 = int(sizeN2/2)
    
    # Number of edge maps, related to sliding windows size
    N_iter = int((M1/sizeM1)*(N1/sizeN1))
    
    Emax1 = np.zeros((N_iter))
    Emax2 = np.zeros((N_iter))
    Emax3 = np.zeros((N_iter))
    
    
    count = 0
    
    # Sliding windows index of level 1
    x1 = 0
    y1 = 0
    # Sliding windows index of level 2
    x2 = 0
    y2 = 0
    # Sliding windows index of level 3
    x3 = 0
    y3 = 0
    
    # Sliding windows limit on horizontal dimension
    Y_limit = N1-sizeN1
    
    while count < N_iter:
        # Get the maximum value of slicing windows over edge maps 
        # in each level
        Emax1[count] = np.max(E1[x1:x1+sizeM1,y1:y1+sizeN1])
        Emax2[count] = np.max(E2[x2:x2+sizeM2,y2:y2+sizeN2])
        Emax3[count] = np.max(E3[x3:x3+sizeM3,y3:y3+sizeN3])
        
        # if sliding windows ends horizontal direction
        # move along vertical direction and resets horizontal
        # direction
        if y1 == Y_limit:
            x1 = x1 + sizeM1
            y1 = 0
            
            x2 = x2 + sizeM2
            y2 = 0
            
            x3 = x3 + sizeM3
            y3 = 0
            
            count += 1
        
        # windows moves along horizontal dimension
        else:
                
            y1 = y1 + sizeN1
            y2 = y2 + sizeN2
            y3 = y3 + sizeN3
            count += 1
    
    # Step 3
    EdgePoint1 = Emax1 > threshold;
    EdgePoint2 = Emax2 > threshold;
    EdgePoint3 = Emax3 > threshold;
    
    # Rule 1 Edge Pojnts
    EdgePoint = EdgePoint1 + EdgePoint2 + EdgePoint3
    
    n_edges = EdgePoint.shape[0]
    
    # Rule 2 Dirak-Structure or Astep-Structure
    DAstructure = (Emax1[EdgePoint] > Emax2[EdgePoint]) * (Emax2[EdgePoint] > Emax3[EdgePoint]);
    
    # Rule 3 Roof-Structure or Gstep-Structure
    
    RGstructure = np.zeros((n_edges))

    for i in range(n_edges):
    
        if EdgePoint[i] == 1:
        
            if Emax1[i] < Emax2[i] and Emax2[i] < Emax3[i]:
            
                RGstructure[i] = 1
                
    # Rule 4 Roof-Structure
    
    RSstructure = np.zeros((n_edges))

    for i in range(n_edges):
    
        if EdgePoint[i] == 1:
        
            if Emax2[i] > Emax1[i] and Emax2[i] > Emax3[i]:
            
                RSstructure[i] = 1

    # Rule 5 Edge more likely to be in a blurred image 

    BlurC = np.zeros((n_edges));

    for i in range(n_edges):
    
        if RGstructure[i] == 1 or RSstructure[i] == 1:
        
            if Emax1[i] < threshold:
            
                BlurC[i] = 1                        
        
    # Step 6
    Per = np.sum(DAstructure)/np.sum(EdgePoint)
    
    # Step 7
    if (np.sum(RGstructure) + np.sum(RSstructure)) == 0:
        
        BlurExtent = 100
    else:
        BlurExtent = np.sum(BlurC) / (np.sum(RGstructure) + np.sum(RSstructure))
    
    return Per, BlurExtent
'''
def find_images(input_dir):
    extensions = [".jpg", ".png", ".jpeg"]

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if os.path.splitext(file)[1].lower() in extensions:
                yield os.path.join(root, file)
                
                
if __name__ == '__main__':                
    parser = argparse.ArgumentParser(description='run Haar Wavelet blur detection on a folder')
    parser.add_argument('-i', '--input_dir', dest="input_dir", type=str, required=True, help="directory of images")
    parser.add_argument('-s', '--save_path', dest='save_path', type=str, help="path to save output")
    parser.add_argument("-t", "--threshold", dest='threshold', type=float, default=35, help="blurry threshold")
    parser.add_argument("-d", "--decision", dest='MinZero', type=float, default=0.001, help="MinZero Decision Threshold")
    args = parser.parse_args()
    
    results = []
    
    for input_path in find_images(args.input_dir):
        try:
            I = cv2.imread(input_path)
            per, blurext = blur_detect(I, args.threshold)
            if per < args.MinZero:
                classification = True
            else:
                classification = False
            results.append({"input_path": input_path, "per": per, "blur extent": blurext, "is blur": classification})
            print("{0}, Per: {1:.5f}, blur extent: {2:.3f}, is blur: {3}".format(input_path, per, blurext, classification))
            
            
        except Exception as e:
            print(e)
            pass
        
    if args.save_path:
        
        assert os.path.splitext(args.save_path)[1] == ".json", "You must include the extension .json on the end of the save path"
        
        with open(args.save_path, 'w') as outfile:
            json.dump(results, outfile, sort_keys=True, indent=4)
            outfile.write("\n")
'''


'\ndef find_images(input_dir):\n    extensions = [".jpg", ".png", ".jpeg"]\n\n    for root, dirs, files in os.walk(input_dir):\n        for file in files:\n            if os.path.splitext(file)[1].lower() in extensions:\n                yield os.path.join(root, file)\n                \n                \nif __name__ == \'__main__\':                \n    parser = argparse.ArgumentParser(description=\'run Haar Wavelet blur detection on a folder\')\n    parser.add_argument(\'-i\', \'--input_dir\', dest="input_dir", type=str, required=True, help="directory of images")\n    parser.add_argument(\'-s\', \'--save_path\', dest=\'save_path\', type=str, help="path to save output")\n    parser.add_argument("-t", "--threshold", dest=\'threshold\', type=float, default=35, help="blurry threshold")\n    parser.add_argument("-d", "--decision", dest=\'MinZero\', type=float, default=0.001, help="MinZero Decision Threshold")\n    args = parser.parse_args()\n    \n    results = []\n    \n    for input_path 

In [ ]:
img = cv2.imread("/content/drive/MyDrive/00ad9c99-b81c-4c48-82e7-14ed1c3c07dc.png")
blur_detect(img,35)

(0.0007572889057175312, 0.3272890946502058)

In [ ]:
img = cv2.imread("/content/drive/MyDrive/clean/a67f8b1e-d9df-4dbe-b7e1-ddc90f71ee0a.png")
blur_detect(img, 35)

(0.0025552486187845305, 0.1856430427758657)

In [ ]:
img = cv2.imread("/content/drive/MyDrive/blurry/a73a2c3c-5a4e-4ee1-add8-55a5e84fcacc.png")
blur_detect(img, 35)

(0.0005832037325038881, 0.6243396724775488)

In [ ]:
from os import listdir
from os.path import isfile, join
import re
import numpy as np

mypath = "/content/drive/MyDrive/blurry"
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
extent =[]
blur = []
for file in files:
    img = cv2.imread(mypath +'/' +file)#.astype('int8')
    p, be = blur_detect(img,35)
    print(file, p)
    extent.append(p)
    blur.append(be)
np.median(extent)


0b52ae12-abc6-42bb-94c1-85393d87d8e6.png 0.0058634650286192936
ecb7053f-d167-4017-9241-dd87c4203b75.png 0.0013785497656465398
e98499f9-f452-4760-8188-55e50e4c3158.png 0.00021753317380900588
ff41d066-f970-438d-948e-ac32ebe10076.png 0.010033444816053512
f9418eab-aab6-45ee-972e-cda057428367.png 0.00616827041697508
f2168600-a764-4a58-a1ea-398f07381cc4.png 0.002980505880457548
d0cde6f2-a009-4b0f-8974-0bf3ac785e1d.png 0.003958156629912355
bf89e8fc-313a-4ab0-a1eb-2845492e9ff8.png 0.0
f524f074-74f8-4700-bafb-89d44cd673ed.png 0.002424426092885825
f85b8cdc-de2b-46a9-9c4d-d5ac27ab9019.png 0.0009394572025052192
f37b1986-862a-42aa-bbcd-d527b6a4f8bf.png 0.0022363026462914647
f9d36fee-c504-49ba-8a87-a58737ba2420.png 0.0001694053870913095
f7a45870-21f5-49de-9361-eef9780ad4ad.png 0.005443483829650976
afb76cd9-5692-4c17-9c8c-7492c5d53121.png 0.0022665457842248413
f7109110-95c3-4ba4-b4e2-460213445811.png 0.0008213292898660602
f72857c5-325c-4f84-a1ed-c9ab0976a854.png 0.0009971953879713306
f952f28e-e38f-44

0.0018074262059690023

In [ ]:


mypath = "/content/drive/MyDrive/clean"
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for file in files:
    img = cv2.imread(mypath +'/' +file)#.astype('int8')
    p, be = blur_detect(img,35)
    print(file, p)
    extent.append(p)
np.median(extent)


f0705784-3b8e-4858-bd77-20be0fade413.png 0.004440320763470077
eed95caa-9a1d-4722-8763-6ceb976f4d18.png 0.0029920645245219203
eee4e8fd-e7a0-4c21-98c3-9aa1a39db975.png 0.0037433155080213902
edf5ee53-40f6-499c-892b-975198fc0453.png 0.00625222024866785
dffadfaf-932c-4ecf-b7ec-90acca0dc4e6.png 0.0072997976293726514
af5474c0-2937-49fe-a028-4744bb0f9b5f.png 0.006269813314547376
ee380080-aa03-42b8-ac13-115b2a495f7b.png 0.0031490015360983105
af95917c-37d4-4a8d-ba37-f5fa97c0199e.png 0.0077738515901060075
adfb679a-1c7a-41bf-b3dd-d6ab8d5d6220.png 0.01700747149036571
ad63c388-4d92-4070-93b9-21c783a1c216.png 0.011920865742306391
fc5cb36a-2859-4086-a32e-bd4dad3ff6a8.png 0.0033121722329561135
f929bd7c-bf75-416f-88bc-6b43cec5b902.png 0.0036509162395562734
a67f8b1e-d9df-4dbe-b7e1-ddc90f71ee0a.png 0.0025552486187845305
6894fe60-f4ae-4673-aee0-f4f33f0c578b.png 0.0010424131865268095
51c32cde-75b4-49f1-9ad9-d1deb8dfac3a.png 0.0036256514842510764
d9ca036a-9c26-44ec-860f-f628a34faff3.png 0.007419354838709678


0.003638283861903675

In [ ]:
mypath = "/content/drive/MyDrive/whole_lap_divide/clean"
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
counter = 0 

extent = []
blur =[]
for file in files:
    img = cv2.imread(mypath +'/' +file)#.astype('int8')
    p, be = blur_detect(img,35)
    print(file, p,be)
    extent.append(p)
    blur.append(be)
    counter+=1
    if counter >10:
      break
print(np.median(extent))
print(np.median(blur))


4clean.png 0.0013866128828942392 0.09147206812024256
5clean.png 0.0020199469763918696 0.09908516943692823
6clean.png 0.002687140115163148 0.050780737436032015
7clean.png 0.0031879622545269064 0.05770988379684032
8clean.png 0.004313725490196078 0.04711551331816356
9clean.png 0.006154249050674349 0.05393801153897759
10clean.png 0.006608209429406532 0.03048065650644783
11clean.png 0.005735406576599541 0.034330299089726915
12clean.png 0.00297349709114415 0.07617161716171617
13clean.png 0.002331002331002331 0.0775907417148869
14clean.png 0.003945329012258701 0.19464544138929088
0.0031879622545269064
0.05770988379684032


In [ ]:
mypath = "/content/drive/MyDrive/whole_lap_divide/blurry"
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
counter = 0 

extent = []
blur =[]
for file in files:
    img = cv2.imread(mypath +'/' +file)#.astype('int8')
    p, be = blur_detect(img,35)
    print(file, p,be)
    extent.append(p)
    blur.append(be)
    counter+=1
    if counter >10:
      break
print(np.median(extent))
print(np.median(blur))


396blurry.png 0.0013128528291978469 0.2861337248098225
397blurry.png 0.000792393026941363 0.32743125419181757
398blurry.png 0.001380071763731714 0.48088813940415964
399blurry.png 0.0006975446428571429 0.48992622020431326
400blurry.png 0.001429128231778615 0.20879989461204057
401blurry.png 0.0003913894324853229 0.23541061526274096
436blurry.png 0.014339058999253173 0.39036755386565275
437blurry.png 0.012462006079027355 0.45428156748911463
438blurry.png 0.010625326598153632 0.5652335063009637
439blurry.png 0.011609773002945764 0.6014359351988218
440blurry.png 0.013539088659193479 0.3470597410700359
0.001429128231778615
0.39036755386565275
